In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin .env/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary .env/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


.env/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /opt/conda did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


In [2]:
peft_model_id = "models/llama-13b-without-type"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map={"": 0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, padding_side="left", model_max_length=1024)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"": 0})

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
tokenizer.add_special_tokens({"pad_token": tokenizer.bos_token})

0

In [4]:
import pandas as pd

df = pd.read_json("data/llama_generation/test.json")
data = list(zip(df["question"], df["context"], df["type"]))
data[0]

('This simple household item saves lives. What household item saves lives?',
 'Erin Zaikis often used to think about how she could help change the world, but it wasn\'t until she got very sick with dengue fever that she decided to take action. In high school, Zaikis\' mother was diagnosed with breast cancer, and the teen slipped into depression -- trapped in what she described as a cycle of feeling sorry for herself. In college, she took a course in global poverty, which helped put her own struggles in perspective. And at 19, the Boston native traveled to Mumbai, where she spent a summer living in an orphanage that housed around 100 girls, some of whom had been left in trash cans or abandoned in train stations. "I\'m from a middle-, upper-class town and had never been exposed to extreme poverty, or starvation, or issues like child trafficking," Zaikis said. "It was life-changing." After graduating, Zaikis, now 24, traveled to Thailand, where she worked with organizations that fight chi

In [16]:
PROMPT = {
    "phrase": (
        "Below is an question paired with an context for which generate answer."
        "Write a answer as short as possible (max 5 words). Use only words from context.\n\n"
        "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:\n"
    ),
    "passage": (
        "Below is an question paired with an context for which generate answer."
        "Write a answer which will be from one to three sentences. Use only words from context.\n\n"
        "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:\n"
    ),
    "multi": (
        "Below is an question paired with an context for which generate answer."
        "Write a answer which is multi part that means it contains multiple "
        "phrase or sentences from given text. Use only words from context.\n\n"
        "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:\n"
    ),
}
# PROMPT = (
#         "Below is an question paired with an context for which generate answer. "
#         "Write a answer that with type {type} appropriately completes question.\n\n"
#         "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:\n"
#     )
input_ids = tokenizer(
    [
        PROMPT[typ].format(
            question=clickbait,
            context=context,
        )
        for clickbait, context, typ in data[1:3]
    ],
    return_tensors="pt",
    max_length=1912,
    padding="max_length",
    truncation=True,
).to("cuda:0")
input_ids.pop("token_type_ids")
generated_ids = model.generate(**input_ids, max_new_tokens=50, do_sample=True, num_return_sequences=3)

batch = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

In [18]:
print(df.loc[0, "output"])
print(df.loc[0, "type"])

soap
phrase


In [5]:
import torch
import re
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# PROMPT = {
#     "phrase": (
#         "Below is an question paired with an context for which generate answer."
#         "Write a answer as short as possible (max 5 words). Use only words from context.\n\n"
#         "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:\n"
#     ),
#     "passage": (
#         "Below is an question paired with an context for which generate answer."
#         "Write a answer which will be from one to three sentences. Use only words from context.\n\n"
#         "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:\n"
#     ),
#     "multi": (
#         "Below is an question paired with an context for which generate answer."
#         "Write a answer which is multi part that means it contains multiple "
#         "phrase or sentences from given text. Use only words from context.\n\n"
#         "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:\n"
#     ),
# }
PROMPT = (
    "Below is an question paired with an context for which generate answer. "
    "Write a answer that appropriately completes question.\n\n"
    "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:\n"
)

# PROMPT = (
#         "Below is an question paired with an context for which generate answer. "
#         "Write a answer that with type {type} appropriately completes question.\n\n"
#         "### Question:\n{question}\n\n### Context:\n{context}\n\n### Answer:\n"
#     )
REGRESSOR_PROMPT = "For given question:\n {} \nanswer:\n {} \ncontext:\n{}"
spoilers_generated = []
batch_size = 6
j = 0

for i in tqdm(range(len(data) // batch_size + 1)):
    data_batch = data[i * batch_size : (i + 1) * batch_size]
    input_ids = tokenizer(
        [PROMPT.format(question=clickbait, context=context) for clickbait, context, typ in data_batch],
        return_tensors="pt",
        max_length=1912,
        padding="max_length",
        truncation=True,
    ).to("cuda:0")
    input_ids.pop("token_type_ids")

    with torch.inference_mode():
        generated_ids = model.generate(**input_ids, max_new_tokens=50)

    spoilers_batch = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    for spoiler in tqdm(spoilers_batch):
        generated = spoiler.split("Answer:\n")
        if len(generated) == 1:
            input_ids = tokenizer(
                [PROMPT.format(question=clickbait, context=context[:4500]) for clickbait, context, typ in data[j : j + 1]],
                return_tensors="pt",
                max_length=2048,
                padding="max_length",
                truncation=True,
            ).to("cuda:0")
            input_ids.pop("token_type_ids")

            with torch.inference_mode():
                generated_ids = model.generate(**input_ids, max_new_tokens=50)

            spoilers_batch = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            generated = spoilers_batch[0].split("Answer:\n")
        generated = generated[1]

        if data[j][2] == "multi":
            generated = generated.replace("</s>", "\n")
        else:
            generated = generated.split("</s>")[0]

        spoilers_generated.append(generated)
        j += 1

100%|██████████| 167/167 [1:11:23<00:00, 25.65s/it]


In [6]:
len(spoilers_generated)

1000

In [7]:
import pandas as pd

pd.DataFrame(zip(range(1000), spoilers_generated), columns=["id", "spoiler"]).to_csv(
    "models/llama-13b-without-type/output.csv", index=False
)

In [80]:
pd.read_csv("models/llama-13b-finetuned/output.csv")

,id,spoiler
0,0,soap
1,1,Gwyneth Paltrow
2,2,"They're being so darn romantic together, it's ..."
3,3,JaVale McGee\ninspired to sponsor an Amateur A...
4,4,Alex Owens-Sarno
...,...,...
995,995,‘All Along the Watchtower.’ The Jimi Hendrix c...
996,996,1. Catch it on a good day.\n2. Smooth it out.\...
997,997,full lunar eclipse\nvisible in the United Stat...
998,998,Tami Erin
